# Investigating Vibration of the floor in SPL7
The floor next to the lasertable adjacent to the beamsource was vibrating at certain times during the day.  
A MMA8481 accelerometer was attached to a NodeMCU and attached to the floor, the script below grabs roughly 6s worth of accelerometer data every 6 other seconds.
I found out that it was probably correlated with the Vertiv (Liebert) fan spinning up to 100%, and wanted to check with the accelerometer.

In [1]:
from urllib.error import URLError, HTTPError
import urllib.request
from socket import timeout
import time
import functools
import logging
import numpy as np

def CatchUrllibErrors(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except (URLError, HTTPError, timeout) as err:
            logging.warning('WirelessSensorStationLasertable warning in {0}() : '.format(func.__name__) \
                            +str(err))
            return np.nan
    return wrapper

class ESP8266AccelerometerHTTP:
    def __init__(self, time_offset, ip):
        self.time_offset = time_offset
        
        self.ip = ip
        
        self.verification_string = self.VerifyOperation()
        
        # shape and type of the array of returned data
        self.dtype = 'f4'
        self.shape = (3, )
        
        self.start = False

        self.warnings = []

    def __enter__(self):
        return self

    def __exit__(self, *exc):
        if self.instr:
            self.StopMeasurement()
            
    @CatchUrllibErrors
    def VerifyOperation(self):
        self.StopMeasurement()
        with urllib.request.urlopen("http://"+self.ip+"/STATUS") as response:
            status = response.read().decode()
        return status.split(',')[0]
    
    @CatchUrllibErrors
    def StopMeasurement(self):
        with urllib.request.urlopen("http://"+self.ip+"/STOP") as response:
            stop = response.read().decode()
            
    @CatchUrllibErrors
    def StartMeasurement(self):
        with urllib.request.urlopen("http://"+self.ip+"/START") as response:
            start = response.read().decode()
            
    @CatchUrllibErrors
    def MMA8451Data(self):
        with urllib.request.urlopen("http://"+self.ip+"/MMA8451") as response:
            data = response.read().decode()
        return np.array([[float(di) for di in d.split(",")] for d in data.split(';')[:-1]])
        
    def ReadValues(self):
        if not self.start:
            self.StartMeasurement()
            self.start = True
        data = self.MMA8451Data()
        t = time.time()
        data[:,0] = data[:,0]/1e6 - data[-1,0]/1e6 + (t - self.time_offset)
        return data

In [2]:
import h5py

def create_hdf_grp(fname, grp_name, attrs):
    with h5py.File(fname, 'a') as f:
        grp = f.create_group(grp_name)
        for k,v in attrs.items():
            grp.attrs[k] = v

def create_hdf_dset(fname, grp_name, dset_name, dtype, attrs):
    with h5py.File(fname, 'a') as f:
        grp = f[grp_name]
        dset = grp.create_dataset(dset_name, (0,), dtype = dtype, maxshape = (None,))
        for k,v in attrs.items():
            dset.attrs[k] = v 

def save_to_hdf(fname, grp_name, dset_name, data, dtype):
    with h5py.File(fname, 'a') as f:
        dset = f[grp_name][dset_name]
        dset.resize(dset.shape[0]+d.shape[0], axis = 0)
        dset[-data.shape[0]:] = data

In [3]:
%matplotlib ipympl
# %matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import time

def plot_data(ax, x,y, colors=['C0']):
    if ax.lines:
        for line in ax.lines:
            line.set_xdata(x)
            line.set_ydata(y)
    else:
        for color in colors:
            ax.plot(x, y, color)
    fig.canvas.draw()

fig, ax = plt.subplots()
fig.canvas.layout.height = '600px'
fig.canvas.layout.width = '1000px'

ax.set_xlabel('time')
ax.set_ylabel('accelerometer')
ax.set_xlim(0,1)
ax.set_ylim(0,1);
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
from tqdm import tqdm_notebook as tqdm

measurement_time_minutes = 60
nr_measurements = measurement_time_minutes*60//10

vib = ESP8266AccelerometerHTTP(time.time(), '172.29.133.49')

conv = 4*9.81/(2**14 -1)

fname = 'vibration_measurement.hdf5'
run_name = 'measurement 1'
description = "Measurement of vibration in SPL7 next to laser table adjacent to beam source "+\
              "with an MMA8451 sensor"
names = "time, x acceleration, y acceleration, z acceleration".split(",")
units = "s, m/s^2, m/s^2, m/s^2".split(",")
dtypes = ['float', 'float', 'float', 'float']

attrs_measurement = { "description": description}
attrs = { 'units': f"{dict((name, unit) for name, unit in zip(names, units))}", 'start_time': vib.time_offset}

data_type = np.dtype([(name.strip(""), eval(ty)) for name, ty in zip(names, dtypes)])

create_hdf_grp(fname, run_name, attrs_measurement)
create_hdf_dset(fname, run_name, "MMA8451", data_type, attrs = attrs)

for _ in tqdm(range(nr_measurements)):
    d = vib.ReadValues()
    d[:,1:] *= conv
    dhdf = np.array([tuple(di) for di in list(d)], dtype = data_type)
    norm = np.linalg.norm(d[:,1:], axis = 1)
    offset = norm.ptp()*0.1
    ax.set_xlim([np.min(d[:,0]), np.max(d[:,0])])
    ax.set_ylim([norm.min()-offset, norm.max()+offset]);
    plot_data(ax, d[:,0], norm)
    save_to_hdf(fname, run_name, 'MMA8451', dhdf, data_type)
vib.StopMeasurement()

In [ ]:
from tqdm import tqdm_notebook as tqdm

measurement_time_minutes = 60*12
nr_measurements = measurement_time_minutes*60//13

vib = ESP8266AccelerometerHTTP(time.time(), '172.29.133.49')

conv = 4*9.81/(2**14 -1)

fname = 'vibration_measurement.hdf5'
run_name = 'measurement 2'
description = "Measurement of vibration in SPL7 next to laser table adjacent to beam source "+\
              "with an MMA8451 sensor"
names = "time, x acceleration, y acceleration, z acceleration".split(",")
units = "s, m/s^2, m/s^2, m/s^2".split(",")
dtypes = ['float', 'float', 'float', 'float']

attrs_measurement = { "description": description}
attrs = { 'units': f"{dict((name, unit) for name, unit in zip(names, units))}", 'start_time': vib.time_offset}

data_type = np.dtype([(name.strip(""), eval(ty)) for name, ty in zip(names, dtypes)])

create_hdf_grp(fname, run_name, attrs_measurement)
create_hdf_dset(fname, run_name, "MMA8451", data_type, attrs = attrs)

for _ in tqdm(range(nr_measurements)):
    d = vib.ReadValues()
    d[:,1:] *= conv
    dhdf = np.array([tuple(di) for di in list(d)], dtype = data_type)
    norm = np.linalg.norm(d[:,1:], axis = 1)
    offset = norm.ptp()*0.1
    ax.set_xlim([np.min(d[:,0]), np.max(d[:,0])])
    ax.set_ylim([norm.min()-offset, norm.max()+offset]);
    plot_data(ax, d[:,0], norm)
    save_to_hdf(fname, run_name, 'MMA8451', dhdf, data_type)
vib.StopMeasurement()

In [5]:
from tqdm import tqdm_notebook as tqdm

measurement_time_minutes = 60*12
nr_measurements = measurement_time_minutes*60//13

vib = ESP8266AccelerometerHTTP(time.time(), '172.29.133.49')

conv = 4*9.81/(2**14 -1)

fname = 'vibration_measurement.hdf5'
run_name = 'measurement 3'
description = "Measurement of vibration in SPL7 next to laser table adjacent to beam source "+\
              "with an MMA8451 sensor"
names = "time, x acceleration, y acceleration, z acceleration".split(",")
units = "s, m/s^2, m/s^2, m/s^2".split(",")
dtypes = ['float', 'float', 'float', 'float']

attrs_measurement = { "description": description}
attrs = { 'units': f"{dict((name, unit) for name, unit in zip(names, units))}", 'start_time': vib.time_offset}

data_type = np.dtype([(name.strip(""), eval(ty)) for name, ty in zip(names, dtypes)])

create_hdf_grp(fname, run_name, attrs_measurement)
create_hdf_dset(fname, run_name, "MMA8451", data_type, attrs = attrs)

for _ in tqdm(range(nr_measurements)):
    d = vib.ReadValues()
    d[:,1:] *= conv
    dhdf = np.array([tuple(di) for di in list(d)], dtype = data_type)
    norm = np.linalg.norm(d[:,1:], axis = 1)
    offset = norm.ptp()*0.1
    ax.set_xlim([np.min(d[:,0]), np.max(d[:,0])])
    ax.set_ylim([norm.min()-offset, norm.max()+offset]);
    plot_data(ax, d[:,0], norm)
    save_to_hdf(fname, run_name, 'MMA8451', dhdf, data_type)
vib.StopMeasurement()

OSError: Unable to create file (unable to open file: name = 'vibration_measurement.hdf5', errno = 17, error message = 'File exists', flags = 15, o_flags = 502)

In [7]:
from tqdm import tqdm_notebook as tqdm

measurement_time_minutes = 60*12
nr_measurements = measurement_time_minutes*60//13

vib = ESP8266AccelerometerHTTP(time.time(), '172.29.133.49')

conv = 4*9.81/(2**14 -1)

fname = 'vibration_measurement.hdf5'
run_name = 'measurement 4'
description = "Measurement of vibration in SPL7 next to laser table adjacent to beam source "+\
              "with an MMA8451 sensor"
names = "time, x acceleration, y acceleration, z acceleration".split(",")
units = "s, m/s^2, m/s^2, m/s^2".split(",")
dtypes = ['float', 'float', 'float', 'float']

attrs_measurement = { "description": description}
attrs = { 'units': f"{dict((name, unit) for name, unit in zip(names, units))}", 'start_time': vib.time_offset}

data_type = np.dtype([(name.strip(""), eval(ty)) for name, ty in zip(names, dtypes)])

create_hdf_grp(fname, run_name, attrs_measurement)
create_hdf_dset(fname, run_name, "MMA8451", data_type, attrs = attrs)

for _ in tqdm(range(nr_measurements)):
    d = vib.ReadValues()
    d[:,1:] *= conv
    dhdf = np.array([tuple(di) for di in list(d)], dtype = data_type)
    norm = np.linalg.norm(d[:,1:], axis = 1)
    offset = norm.ptp()*0.1
    ax.set_xlim([np.min(d[:,0]), np.max(d[:,0])])
    ax.set_ylim([norm.min()-offset, norm.max()+offset]);
    plot_data(ax, d[:,0], norm)
    save_to_hdf(fname, run_name, 'MMA8451', dhdf, data_type)
vib.StopMeasurement()

In [4]:
%matplotlib ipympl
# %matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import time

def plot_data(ax, x,y, colors=['C0']):
    if ax.lines:
        for line in ax.lines:
            line.set_xdata(x)
            line.set_ydata(y)
    else:
        for color in colors:
            ax.plot(x, y, color)
    fig.canvas.draw()

fig, ax = plt.subplots()
fig.canvas.layout.height = '600px'
fig.canvas.layout.width = '1000px'

ax.set_xlabel('time')
ax.set_ylabel('accelerometer')
ax.set_xlim(0,1)
ax.set_ylim(0,1);
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Analyzing data
First two measurements the timestamping went wrong, and the fan of the Vertiv wasn't spinning up anyway.
Only looking at last two measurements.

In [46]:
data_hdf = []
with h5py.File(fname) as f:
    for m in [f'measurement {i}' for i in range(3,5)]:
        dset = f[m]['MMA8451']
        data_hdf.append((m, dset.attrs['start_time'], dset.value))

In [47]:
data = []
for d in data_hdf:
    d_tmp = d[2]
    d_tmp['time'] += d[1]
    data.append(d_tmp)

In [89]:
d = data[0]
norm1 = np.linalg.norm(d.view((float, len(d.dtype.names)))[:,1:], axis = 1)
d0 = datetime.fromtimestamp(d['time'][0])
times1 = [d0+timedelta(seconds = dt) for dt in d['time']-d['time'][0]]

d = data[1]
norm2 = np.linalg.norm(d.view((float, len(d.dtype.names)))[:,1:], axis = 1)
d0 = datetime.fromtimestamp(d['time'][0])
times2 = [d0+timedelta(seconds = dt) for dt in d['time']-d['time'][0]]

In [93]:
fig, ax = plt.subplots(figsize = (12,8))
ax.plot(times1[::5], norm1[::5])
ax.plot(times2[::5], norm2[::5])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Looks like there is some structure here, but going to try to filter it a bit by taking the peak-to-peak over every 1000 samples (which is the nr of samples I grab every 6 other seconds)

## Filtering data with ptp

In [146]:
t_mean = d['time'].reshape(norm.size//1000, 1000).mean(axis = 1)
norm_ptp= norm.reshape(norm.size//1000, 1000).ptp(axis = 1)

In [155]:
fig, ax = plt.subplots(figsize = (12,8))

for d in data:
    norm = np.linalg.norm(d.view((float, len(d.dtype.names)))[:,1:], axis = 1)
    t_mean = d['time'].reshape(norm.size//1000, 1000).mean(axis = 1)
    norm_ptp= norm.reshape(norm.size//1000, 1000).ptp(axis = 1) 
    d0 = datetime.fromtimestamp(d['time'][0])
    times = [d0+timedelta(seconds = dt) for dt in t_mean - t_mean[0]]
    
    ax.plot(times, norm_ptp)
ax.set_xlabel('time')
ax.set_ylabel('accelerometer "noise" [m/s^2]');

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<img src="vertiv_params.png" />

Now it seems pretty clear the vibration correlates with the Vertiv fan spinning up.  
Vertiv sent some engineers on the 3rd of November to make sure the unit was operating properly after a sudden 5C temperature drop a few weeks back, and while they didn't find anything wrong with the unit (the issue was probably the hot water supply), the unit itself noticable shakes less after their visit, where they opened the unit up as wel